In [1]:
# !pip install numpy==1.21.5  # Needed by imaginaryNLP

In [2]:
from datasets import load_dataset

dataset = load_dataset("diwank/imaginary-nlp-dataset")
dataset_with_length = dataset.map(lambda row: dict(length=len(" ".join(row["dialog"]).split(" "))))

# Remove dialogs that are too long
MAX_WORDS_PER_DIALOG = 300
dataset = dataset_with_length.filter(lambda row: row["length"] <= MAX_WORDS_PER_DIALOG).remove_columns(["length"])

/lab/github.com/Justus-Jonas/imaginaryNLP/.venv/lib/python3.10/site-packages/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/982313 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/106237 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/118790 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/106237 [00:00<?, ? examples/s]

Map:   0%|          | 0/118790 [00:00<?, ? examples/s]

Filter:   0%|          | 0/982313 [00:00<?, ? examples/s]

Filter:   0%|          | 0/106237 [00:00<?, ? examples/s]

Filter:   0%|          | 0/118790 [00:00<?, ? examples/s]

In [3]:
import torch
from imaginaryNLP.trainer import ImaginaryEmbeddingTrainer

base_model_name_or_path = "thenlper/gte-large"
batch_size = 16
observation_window = 5
speaker_token = True
num_epochs = 3
warmup_steps = 10000
model_args = dict(
    # device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

trainer = ImaginaryEmbeddingTrainer(
    base_model_name_or_path=base_model_name_or_path,
    batch_size=batch_size,
    observation_window=observation_window,
    speaker_token=speaker_token,
    num_epochs=num_epochs,
    warmup_steps=warmup_steps,
    model_args=model_args,
)

In [4]:
# !aria2c https://sbert.net/datasets/AllNLI.tsv.gz
# !mv AllNLI.tsv.gz ./datasets/AllNLI.tsv.gz

In [5]:
trainer.generate_datasets(
    dataset["train"]["dialog"],
    dataset["validation"]["dialog"],
    dataset["test"]["dialog"],
)

In [ ]:
model_save_path = "./gte-large-imaginary_nlp-spt"
lr = 2e-5
weight_decay: float = 0.01

trainer.train(
    model_save_path,
    batch_size=batch_size,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    optimizer_params=dict(lr=lr),
    weight_decay=weight_decay,
    checkpoint_save_steps=500,
    checkpoint_save_total_limit=3,
    use_amp=False,
)

/lab/github.com/Justus-Jonas/imaginaryNLP/.venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/58880 [00:00<?, ?it/s]